In [8]:
import os
import csv
import json
import sys
import numpy as np
import random

import librosa

In [9]:
sound_emotion_path = "../../databases/TESS_and_RAVDESS/"
train_data = {
    "mfcc": [],
    "label": [],
    "emotion": []
}

validation_data = {
    "mfcc": [],
    "label": [],
    "emotion": []
}

test_data = {
    "mfcc": [],
    "label": [],
    "emotion": []
}

max_value = 0
min_value = 0

In [10]:
def process_sound(filename, dirpath, emotion, label, sample_rate, n_mfcc, n_fft, hop_length):
    file_path = os.path.join(dirpath, filename)
    # sample rate is per-second, signal.size/sample rate = time
    signal, sr = librosa.load(file_path, sr=sample_rate)
    # Option to round this so we get more samples, but having consistent 1 second samples is important too
    intervals = int(signal.size / sample_rate)
    for x in range(intervals):
        start = x * sample_rate
        stop = start + sample_rate
        mfcc = np.mean(librosa.feature.mfcc(signal[start:stop], sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc),axis=0)      
        # max_value = np.amax(mfcc)
        # min_value = np.amin(mfcc)
        # mfcc = 2*((mfcc-min_value)/(max_value-min_value))-1        
        # global max_value, min_value

        randint = random.random()
        if randint < 0.8:
            # max_value = max(max_value, np.amax(mfcc))
            # min_value = min(min_value, np.amin(mfcc))

            train_data["mfcc"].append(mfcc.tolist())
            train_data["label"].append(label)
            train_data["emotion"].append(emotion)
        elif randint >= 0.8 and randint < 0.9:
            # max_value = max(max_value, np.amax(mfcc))
            # min_value = min(min_value, np.amin(mfcc))

            validation_data["mfcc"].append(mfcc.tolist())
            validation_data["label"].append(label)
            validation_data["emotion"].append(emotion)            
        else:       
            test_data["mfcc"].append(mfcc.tolist())
            test_data["label"].append(label)
            test_data["emotion"].append(emotion)             

In [11]:
def save_mfcc(dataset_path, n_mfcc=13, n_fft=1024, hop_length=256, sample_rate=22050):
    # count: 0=Angry, 1=Disgusted, 2=Fearful, 3=Happy, 4=Netural, 5=Sad, 6=Surprised
    for count, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # print(filenames)
        # print(dirnames)
        print(dirpath)
        if dirpath is not sound_emotion_path:
            if "Angry" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "anger_disgust", 0, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Disgusted" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "anger_disgust", 0, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Suprised" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "surprised_fear", 4, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Fearful" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "surprised_fear", 4, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Happy" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "joy", 1, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Sad" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "sadness", 3, sample_rate, n_mfcc, n_fft, hop_length)
            elif "Neutral" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "neutral", 2, sample_rate, n_mfcc, n_fft, hop_length)
            else:
                continue

save_mfcc(sound_emotion_path)

../../databases/TESS_and_RAVDESS/
../../databases/TESS_and_RAVDESS/Emotions
../../databases/TESS_and_RAVDESS/Emotions\Angry
../../databases/TESS_and_RAVDESS/Emotions\Disgusted
../../databases/TESS_and_RAVDESS/Emotions\Fearful
../../databases/TESS_and_RAVDESS/Emotions\Happy
../../databases/TESS_and_RAVDESS/Emotions\Neutral
../../databases/TESS_and_RAVDESS/Emotions\Sad
../../databases/TESS_and_RAVDESS/Emotions\Suprised


In [12]:
print(max_value, min_value)

0 0


In [13]:
# nparray_list = np.array(train_data["mfcc"])
# normalized_nparray = 2*((nparray_list-min_value)/(max_value-min_value))-1
# train_data["mfcc"] = normalized_nparray.tolist()
# print("--------done-----------")
# nparray_list = np.array(validation_data["mfcc"])
# normalized_nparray = 2*((nparray_list-min_value)/(max_value-min_value))-1
# validation_data["mfcc"] = normalized_nparray.tolist()
# print("--------done-----------")
# nparray_list = np.array(test_data["mfcc"])
# normalized_nparray = 2*((nparray_list-min_value)/(max_value-min_value))-1
# test_data["mfcc"] = normalized_nparray.tolist()
# print("--------done-----------")

In [14]:
with open("./tess_ravdess_train_norm.json", "w") as fp:
    json.dump(train_data, fp, indent=4)
with open("./tess_ravdess_validation_norm.json", "w") as fp:
    json.dump(validation_data, fp, indent=4)
with open("./tess_ravdess_test_norm.json", "w") as fp:
    json.dump(test_data, fp, indent=4)